***Proyecto Esperanza de vida ***

El dataset Esperanza_de_vida2000-2015.csv contiene variables relacionadas con la esperanza de vida y factores socioeconómicos, de salud y demográficos de diferentes países entre 2000 y 2015. Con este dataset, podemos predecir varias cosas más allá de la esperanza de vida, que ya se aborda en el código.


Este dataset es muy versátil y permite predecir una amplia gama de variables relacionadas con la salud, el desarrollo y las condiciones socioeconómicas. Puedes enfocarte en predicciones supervisadas (regresión, clasificación), análisis no supervisado (clustering) o incluso análisis causales. ¿Te interesa profundizar en alguna de estas ideas? Puedo ayudarte a implementarla.




# 1. Instalación de Dependencias
Primero, instalamos todas las dependencias necesarias. Estas líneas deben ejecutarse solo una vez en Google Colab.

In [1]:
# Instalar dependencias necesarias
!pip install --upgrade gradio
!pip install matplotlib seaborn
!pip install gradio requests
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install huggingface_hub[hf_xet]
!pip install folium geopandas
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cagwp_dt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-cagwp_dt
  Resolved https://github.com/huggingface/transformers.git to commit 1897a02d83dd7d3cae62b1a4882df61dcc15f3b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.

# 2. Importación de Bibliotecas
Importamos todas las bibliotecas necesarias para el proyecto.


In [2]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import gradio as gr
import torch
from transformers import pipeline
from huggingface_hub import login
from google.colab import userdata
import requests
import zipfile
import os
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go

# 3. Verificación de GPU
Verificamos si hay una GPU disponible para acelerar el modelo de lenguaje.

In [3]:
# Verificar GPU
if torch.cuda.is_available():
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    print("No se detectó GPU. El modelo se ejecutará en CPU, lo que puede ser más lento.")

GPU disponible: NVIDIA A100-SXM4-40GB


# 4. Autenticación en Hugging Face
Nos autenticamos en Hugging Face para descargar el modelo zephyr-7b-beta.

In [4]:
# Autenticación en Hugging Face
api_key = userdata.get('HF_TOKEN')  # Asegúrate de tener el token en Colab Secrets
login(api_key)

# 5. Carga del Modelo de Lenguaje
Cargamos el modelo zephyr-7b-beta para responder preguntas.

In [5]:
# Cargar el modelo de lenguaje
try:
    pipe = pipeline(
        "text-generation",
        model="HuggingFaceH4/zephyr-7b-beta",
        torch_dtype=torch.bfloat16,
        device_map="auto",
        model_kwargs={"low_cpu_mem_usage": True}
    )
except Exception as e:
    print(f"Error al cargar el modelo: {str(e)}")
    pipe = None

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


# 6. Carga y Limpieza del Dataset
Cargamos y limpiamos el dataset Esperanza_de_vida2000-2015.csv.

In [6]:
# Cargar y limpiar el dataset
df = pd.read_csv('/content/Esperanza_de_vida2000-2015.csv', sep=';')
df.columns = df.columns.str.strip()

# Convertir columnas problemáticas a numérico
for col in ['GDP', 'percentage expenditure']:
    df[col] = pd.to_numeric(df[col].str.replace('.', ''), errors='coerce')

# Limpiar datos
df = df.drop_duplicates()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# 7. Preparación de Datos para el Modelo
Preparamos los datos para entrenar el modelo de regresión lineal.

In [7]:
# Preparar datos
df_with_country = df.copy()  # Copia para análisis por países
df = df.drop(['Country'], axis=1, errors='ignore')
df = pd.get_dummies(df, columns=['Status'], drop_first=True)

X = df.drop('Life expectancy', axis=1)
y = df['Life expectancy']

# 8. División y Entrenamiento del Modelo
Dividimos los datos y entrenamos el modelo de regresión lineal.

In [8]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluar modelo
mse = mean_squared_error(y_test, model.predict(X_test))
r2 = r2_score(y_test, model.predict(X_test))

# Guardar coeficientes
coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': model.coef_
}).sort_values(by='Coeficiente', ascending=False)

# 9. Definición de Preguntas Predefinidas
Definimos preguntas predefinidas para la pestaña de preguntas.

In [9]:
# Preguntas predefinidas
predefined_questions = [
    "Selecciona una pregunta",
    "¿Qué secretos esconden los factores que predicen nuestra esperanza de vida?",
    "¿Cuáles son los verdaderos titanes que mueven los hilos de cuántos años vivimos?",
    "Si tu país está por debajo de los 65 años de esperanza de vida, ¿podría un empujón al gasto en salud cambiar el juego?",
    "¿Cómo juegan las tasas de mortalidad infantil y adulta en el tablero de la esperanza de vida?",
    "¿Comer, vivir y beber influyen en cuántos años celebramos?",
    "¿Es la esperanza de vida amiga o enemiga de nuestros hábitos diarios como el alcohol?",
    "¿Puede la educación ser la clave mágica para vivir más?",
    "¿Es el alcohol un ladrón silencioso de años o un compañero inofensivo?",
    "¿Vivir apretados en un país superpoblado acorta nuestros días?",
    "¿Son las vacunas el escudo secreto que alarga nuestras vidas?",
    "¿Qué es el polio?"
]

# 10. Descarga del Shapefile para el Mapa
Descargamos el shapefile de Natural Earth para el mapa interactivo.

In [10]:
# Descargar shapefile
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
try:
    response = requests.get(url, stream=True)
    response.raise_for_status()
    zip_path = "/content/ne_110m_admin_0_countries.zip"
    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("/content/ne_110m_admin_0_countries")
    world = gpd.read_file("/content/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
    world = world.rename(columns={'NAME': 'Country', 'ISO_A3': 'iso_alpha'})
except Exception as e:
    print(f"Error al descargar shapefile: {e}")
    world = px.data.gapminder()[['country', 'iso_alpha']].drop_duplicates()
    world = world.rename(columns={'country': 'Country'})

# 11. Preparación de Datos para el Mapa
Preparamos los datos para el mapa, incluyendo el mapeo de nombres de países.

In [11]:
# Calcular esperanza de vida promedio por país
country_avg = df_with_country.groupby('Country')['Life expectancy'].mean().reset_index()
country_avg = country_avg.sort_values(by='Life expectancy', ascending=False)

# Top 10 países
top_10_countries = country_avg.head(10)

# Diccionario de mapeo de nombres
country_mapping = {
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Iran (Islamic Republic of)': 'Iran',
    'Republic of Korea': 'South Korea',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Russian Federation': 'Russia',
    'Viet Nam': 'Vietnam',
    'United Republic of Tanzania': 'Tanzania',
    'Congo': 'Dem. Rep. Congo',
    "Côte d'Ivoire": 'Ivory Coast',
    "Cote d'Ivoire": 'Ivory Coast',
    'Syrian Arab Republic': 'Syria',
    'Lao People\'s Democratic Republic': 'Laos',
    'Democratic People\'s Republic of Korea': 'North Korea',
    'Korea, North': 'North Korea',
    'Republic of Moldova': 'Moldova',
    'Brunei Darussalam': 'Brunei',
    'Micronesia, Federated States of': 'Micronesia',
    'The former Yugoslav republic of Macedonia': 'North Macedonia',
    'Macedonia': 'North Macedonia',
    'Kyrgyzstan': 'Kyrgyz Republic',
    'Swaziland': 'Eswatini',
    'Timor-Leste': 'East Timor',
    'East Timor': 'East Timor',
    'Cabo Verde': 'Cape Verde',
    'Cape Verde': 'Cape Verde',
    'Dominican Republic': 'Dominican Rep.',
    'Trinidad and Tobago': 'Trinidad and Tobago',
    'Antigua and Barbuda': 'Antigua and Barbuda',
    'Saint Lucia': 'St. Lucia',
    'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'St. Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'Bosnia and Herzegovina': 'Bosnia and Herzegovina',
    'Sao Tome and Principe': 'São Tomé and Principe',
    'Central African Republic': 'Central African Rep.',
    'Equatorial Guinea': 'Eq. Guinea',
    'Papua New Guinea': 'Papua New Guinea',
    'South Sudan': 'S. Sudan',
    'Guinea-Bissau': 'Guinea-Bissau',
    'Sierra Leone': 'Sierra Leone',
    'Solomon Islands': 'Solomon Is.',
    'Saudi Arabia': 'Saudi Arabia',
    'United Arab Emirates': 'United Arab Emirates',
    'Czech Republic': 'Czechia',
    'Slovakia': 'Slovak Republic',
    'Serbia': 'Serbia',
    'Montenegro': 'Montenegro',
    'Kosovo': 'Kosovo',
    'Albania': 'Albania',
    'Croatia': 'Croatia',
    'Slovenia': 'Slovenia',
    'Estonia': 'Estonia',
    'Latvia': 'Latvia',
    'Lithuania': 'Lithuania',
    'Belarus': 'Belarus',
    'Ukraine': 'Ukraine',
    'Armenia': 'Armenia',
    'Azerbaijan': 'Azerbaijan',
    'Georgia': 'Georgia',
    'Kazakhstan': 'Kazakhstan',
    'Uzbekistan': 'Uzbekistan',
    'Turkmenistan': 'Turkmenistan',
    'Tajikistan': 'Tajikistan',
    'Afghanistan': 'Afghanistan',
    'Pakistan': 'Pakistan',
    'India': 'India',
    'Bangladesh': 'Bangladesh',
    'Nepal': 'Nepal',
    'Bhutan': 'Bhutan',
    'Sri Lanka': 'Sri Lanka',
    'Maldives': 'Maldives',
    'Myanmar': 'Myanmar',
    'Burma': 'Myanmar',
    'Thailand': 'Thailand',
    'Cambodia': 'Cambodia',
    'Malaysia': 'Malaysia',
    'Indonesia': 'Indonesia',
    'Philippines': 'Philippines',
    'Singapore': 'Singapore',
    'China': 'China',
    'Mongolia': 'Mongolia',
    'Japan': 'Japan',
    'Australia': 'Australia',
    'New Zealand': 'New Zealand',
    'Fiji': 'Fiji',
    'Vanuatu': 'Vanuatu',
    'Samoa': 'Samoa',
    'Tonga': 'Tonga',
    'Kiribati': 'Kiribati',
    'Tuvalu': 'Tuvalu',
    'Nauru': 'Nauru',
    'Marshall Islands': 'Marshall Is.',
    'Marshall Is.': 'Marshall Is.',
    'Palau': 'Palau',
    'Egypt': 'Egypt',
    'Algeria': 'Algeria',
    'Morocco': 'Morocco',
    'Tunisia': 'Tunisia',
    'Libya': 'Libya',
    'Sudan': 'Sudan',
    'Eritrea': 'Eritrea',
    'Ethiopia': 'Ethiopia',
    'Somalia': 'Somalia',
    'Djibouti': 'Djibouti',
    'Kenya': 'Kenya',
    'Uganda': 'Uganda',
    'Rwanda': 'Rwanda',
    'Burundi': 'Burundi',
    'Zambia': 'Zambia',
    'Malawi': 'Malawi',
    'Mozambique': 'Mozambique',
    'Zimbabwe': 'Zimbabwe',
    'Botswana': 'Botswana',
    'Namibia': 'Namibia',
    'South Africa': 'South Africa',
    'Lesotho': 'Lesotho',
    'Angola': 'Angola',
    'Cameroon': 'Cameroon',
    'Chad': 'Chad',
    'Niger': 'Niger',
    'Mali': 'Mali',
    'Burkina Faso': 'Burkina Faso',
    'Mauritania': 'Mauritania',
    'Senegal': 'Senegal',
    'Gambia': 'Gambia',
    'Guinea': 'Guinea',
    'Liberia': 'Liberia',
    'Ghana': 'Ghana',
    'Togo': 'Togo',
    'Benin': 'Benin',
    'Nigeria': 'Nigeria',
    'Gabon': 'Gabon',
    'Comoros': 'Comoros',
    'Madagascar': 'Madagascar',
    'Seychelles': 'Seychelles',
    'Mauritius': 'Mauritius',
    'Jordan': 'Jordan',
    'Lebanon': 'Lebanon',
    'Iraq': 'Iraq',
    'Kuwait': 'Kuwait',
    'Qatar': 'Qatar',
    'Bahrain': 'Bahrain',
    'Oman': 'Oman',
    'Yemen': 'Yemen',
    'Turkey': 'Turkey',
    'Cyprus': 'Cyprus',
    'Israel': 'Israel',
    'Palestine': 'Palestine',
    'Greece': 'Greece',
    'Bulgaria': 'Bulgaria',
    'Romania': 'Romania',
    'Hungary': 'Hungary',
    'Poland': 'Poland',
    'Austria': 'Austria',
    'Switzerland': 'Switzerland',
    'Belgium': 'Belgium',
    'Netherlands': 'Netherlands',
    'Luxembourg': 'Luxembourg',
    'Denmark': 'Denmark',
    'Norway': 'Norway',
    'Sweden': 'Sweden',
    'Finland': 'Finland',
    'Iceland': 'Iceland',
    'Ireland': 'Ireland',
    'Portugal': 'Portugal',
    'Spain': 'Spain',
    'France': 'France',
    'Italy': 'Italy',
    'Germany': 'Germany',
    'Malta': 'Malta',
    'Andorra': 'Andorra',
    'Monaco': 'Monaco',
    'Canada': 'Canada',
    'Mexico': 'Mexico',
    'Guatemala': 'Guatemala',
    'Belize': 'Belize',
    'El Salvador': 'El Salvador',
    'Honduras': 'Honduras',
    'Nicaragua': 'Nicaragua',
    'Costa Rica': 'Costa Rica',
    'Panama': 'Panama',
    'Cuba': 'Cuba',
    'Haiti': 'Haiti',
    'Jamaica': 'Jamaica',
    'Bahamas, The': 'Bahamas',
    'Barbados': 'Barbados',
    'Dominica': 'Dominica',
    'Grenada': 'Grenada',
    'St. Kitts and Nevis': 'St. Kitts and Nevis',
    'Colombia': 'Colombia',
    'Ecuador': 'Ecuador',
    'Peru': 'Peru',
    'Chile': 'Chile',
    'Argentina': 'Argentina',
    'Uruguay': 'Uruguay',
    'Paraguay': 'Paraguay',
    'Brazil': 'Brazil',
    'Curacao': 'Curaçao',
}

# Aplicar mapeo
country_avg['Country'] = country_avg['Country'].replace(country_mapping)

# Unir datos con shapefile
world_merged = world.merge(country_avg, on='Country', how='left')

# Verificar mapeo
missing_countries = set(country_avg['Country']) - set(world['Country'])
if missing_countries:
    print(f"Países no encontrados en el shapefile: {missing_countries}")
else:
    print("Todos los países del dataset han sido mapeados correctamente.")

mapped_countries = set(country_avg['Country']).intersection(set(world['Country']))
print(f"Países mapeados correctamente: {len(mapped_countries)} de {len(country_avg)}")
print(f"Ejemplo de países mapeados: {list(mapped_countries)[:10]}")

Países no encontrados en el shapefile: {'Saint Kitts and Nevis', 'Maldives', 'Grenada', 'Slovak Republic', 'Malta', 'Singapore', 'Kiribati', 'Ivory Coast', 'Seychelles', 'Mauritius', 'United States', 'Cook Islands', 'San Marino', 'St. Lucia', 'Tonga', 'Marshall Is.', 'Tuvalu', 'Antigua and Barbuda', 'São Tomé and Principe', 'Barbados', 'Micronesia (Federated States of)', 'Palau', 'Kyrgyz Republic', 'Niue', 'St. Vincent and the Grenadines', 'Samoa', 'Bahrain', 'East Timor', 'Dominica', 'Eswatini', 'Nauru', 'Bosnia and Herzegovina', 'Democratic Republic of the Congo', 'Monaco', 'Cape Verde', 'Comoros'}
Países mapeados correctamente: 157 de 193
Ejemplo de países mapeados: ['Trinidad and Tobago', 'Sudan', 'Bolivia', 'Iraq', 'Uzbekistan', 'Madagascar', 'India', 'Saudi Arabia', 'Mongolia', 'Kuwait']


# 12. Definición de Funciones
Definimos todas las funciones necesarias para la interfaz.

In [12]:
# Función para consultar el modelo de lenguaje
def query_huggingface_pipeline(question):
    if pipe is None:
        return "❌ Error: No se pudo cargar el modelo de lenguaje."
    try:
        messages = [
            {"role": "system", "content": "Eres un chatbot que proporciona respuestas sobre datos de esperanza de vida."},
            {"role": "user", "content": question}
        ]
        prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        outputs = pipe(prompt, max_new_tokens=50, do_sample=False, temperature=0.7, top_k=40, top_p=0.9)
        response = outputs[0]["generated_text"].split("<|assistant|>")[1].strip()
        return response
    except Exception as e:
        return f"❌ Error al procesar la pregunta: {str(e)}"

# Función para generar histograma
def plot_life_expectancy_histogram():
    plt.figure(figsize=(8, 6), facecolor='#FFFFFF')
    sns.histplot(df['Life expectancy'], bins=30, kde=True, color='#4285F4')
    plt.title('Distribución de la Esperanza de Vida', fontsize=14, fontfamily='Roboto', color='#F28C38')
    plt.xlabel('Esperanza de Vida (años)', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.ylabel('Frecuencia', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.gca().set_facecolor('#FFFFFF')
    return plt

# Función para generar boxplot
def plot_boxplot():
    plt.figure(figsize=(10, 6), facecolor='#FFFFFF')
    sns.boxplot(data=df[['Life expectancy', 'Adult Mortality', 'Schooling']], palette=['#4285F4', '#F28C38', '#F4B400'])
    plt.title('Distribución de Variables Clave', fontsize=14, fontfamily='Roboto', color='#F28C38')
    plt.ylabel('Valores', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.gca().set_facecolor('#FFFFFF')
    return plt

# Función para estadísticas del dataset
def get_dataset_stats():
    stats_df = df.describe()
    stats_html = stats_df.to_html(classes='table table-striped', border=0)
    stats_html = f"""
    <style>
        .table {{ width: 100%; border-collapse: collapse; font-family: Roboto, sans-serif; background-color: #FFFFFF; color: #0F9D58; }}
        .table th, .table td {{ padding: 10px; text-align: center; border-bottom: 1px solid #4285F4; }}
        .table th {{ background-color: #4285F4; color: #FFFFFF; }}
        .table tr:nth-child(even) {{ background-color: #F5F5F5; }}
        .table tr:hover {{ background-color: #F4B400; }}
    </style>
    {stats_html}
    """
    explanation = """
    ### 💻 ¿Qué significan estas estadísticas?
    - **count**: Número de datos disponibles.
    - **mean**: Promedio de los valores.
    - **std**: Desviación estándar (dispersión).
    - **min/max**: Valores mínimo y máximo.
    - **25%/50%/75%**: Cuartiles (rango de datos).
    """
    hist_explanation = """
    ### ⚡ Histograma de la Esperanza de Vida
    Muestra la distribución de la esperanza de vida. Una curva simétrica indica uniformidad; un sesgo indica más países con valores bajos o altos.
    """
    boxplot_explanation = """
    ### 📊 Boxplot de Variables Clave
    - **Caja**: 50% central de los datos.
    - **Línea**: Mediana.
    - **Bigotes**: Rango no atípico.
    - **Puntos**: Outliers.
    """
    return stats_html, explanation, plot_life_expectancy_histogram(), hist_explanation, plot_boxplot(), boxplot_explanation

# Función para gráfico de coeficientes
def plot_coefficients():
    plt.figure(figsize=(10, 6), facecolor='#FFFFFF')
    sns.barplot(x='Coeficiente', y='Variable', data=coef_df, palette=['#4285F4', '#F28C38', '#F4B400'])
    plt.title('Importancia de las Variables (Coeficientes)', fontsize=14, fontfamily='Roboto', color='#F28C38')
    plt.xlabel('Coeficiente', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.ylabel('Variable', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.gca().set_facecolor('#FFFFFF')
    return plt

# Función para gráfico de predicciones vs reales
def plot_predictions_vs_actual():
    plt.figure(figsize=(8, 6), facecolor='#FFFFFF')
    plt.scatter(y_test, model.predict(X_test), alpha=0.5, color='#4285F4')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Esperanza de Vida Real', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.ylabel('Esperanza de Vida Predicha', fontsize=12, fontfamily='Roboto', color='#0F9D58')
    plt.title('Predicciones vs Valores Reales', fontsize=14, fontfamily='Roboto', color='#F28C38')
    plt.gca().set_facecolor('#FFFFFF')
    return plt

# Función para detalles del modelo
def get_model_details():
    coef_html = coef_df.to_html(classes='table table-striped', border=0)
    coef_html = f"""
    <style>
        .table {{ width: 50%; border-collapse: collapse; font-family: Roboto, sans-serif; background-color: #FFFFFF; color: #0F9D58; }}
        .table th, .table td {{ padding: 10px; text-align: center; border-bottom: 1px solid #4285F4; }}
        .table th {{ background-color: #4285F4; color: #FFFFFF; }}
        .table tr:nth-child(even) {{ background-color: #F5F5F5; }}
        .table tr:hover {{ background-color: #F4B400; }}
    </style>
    {coef_html}
    """
    metrics = f"Error cuadrático medio (MSE): {mse:.2f}\nCoeficiente de determinación (R²): {r2:.2f}"
    metrics_explanation = """
    ### 💻 ¿Qué significan estas métricas?
    - **MSE**: Promedio de errores al cuadrado. Más bajo = mejor.
    - **R²**: Proporción de varianza explicada. Cerca de 1 = buen ajuste.
    """
    coef_explanation = """
    ### ⚡ ¿Qué significan los coeficientes?
    - **Positivo**: Aumentar la variable incrementa la esperanza de vida.
    - **Negativo**: Aumentar la variable reduce la esperanza de vida.
    - **Magnitud**: Indica la fuerza del impacto.
    """
    scatter_explanation = """
    ### 📊 Predicciones vs Valores Reales
    - Puntos cerca de la línea roja (y=x): Buenas predicciones.
    - Puntos dispersos: Más errores.
    """
    return coef_html, coef_explanation, plot_coefficients(), metrics, metrics_explanation, plot_predictions_vs_actual(), scatter_explanation

# Función para predecir esperanza de vida
def predict_life_expectancy(year, adult_mortality, infant_deaths, alcohol, percentage_expenditure,
                           hepatitis_b, measles, bmi, under_five_deaths, polio, total_expenditure,
                           diphtheria, hiv_aids, gdp, population, thinness_1_19_years,
                           thinness_5_9_years, income_composition, schooling, status_developed):
    input_data = np.array([[
        year, adult_mortality, infant_deaths, alcohol, percentage_expenditure,
        hepatitis_b, measles, bmi, under_five_deaths, polio, total_expenditure,
        diphtheria, hiv_aids, gdp, population, thinness_1_19_years,
        thinness_5_9_years, income_composition, schooling, status_developed
    ]])
    prediction = model.predict(input_data)[0]
    return f"⚡ Esperanza de vida predicha: {prediction:.2f} años"

# Función para análisis por país
def get_country_analysis():
    map_fig = px.choropleth(
        world_merged,
        locations='iso_alpha',
        color='Life expectancy',
        hover_name='Country',
        hover_data={'Life expectancy': True, 'iso_alpha': False},
        color_continuous_scale='YlOrRd',
        title='Esperanza de Vida Promedio por País',
        labels={'Life expectancy': 'Esperanza de Vida (años)'}
    )
    map_fig.update_layout(
        margin={"r":0,"t":50,"l":0,"b":0},
        geo=dict(showframe=False, showcoastlines=True, projection_type='equirectangular'),
        coloraxis_colorbar=dict(title="Esperanza de Vida (años)", thicknessmode="pixels", thickness=15, lenmode="pixels", len=300),
        title_font=dict(family="Roboto", size=20, color="#1A73E8"),
        font=dict(family="Roboto", color="#333333")
    )
    map_fig.update_layout(width=800, height=500)

    top_10_fig = go.Figure(data=[
        go.Bar(
            x=top_10_countries['Life expectancy'],
            y=top_10_countries['Country'],
            orientation='h',
            marker=dict(color=top_10_countries['Life expectancy'], colorscale='YlOrRd'),
            text=top_10_countries['Life expectancy'].round(2),
            textposition='auto',
        )
    ])
    top_10_fig.update_layout(
        title='Top 10 Países con Mayor Esperanza de Vida',
        xaxis_title='Esperanza de Vida (años)',
        yaxis_title='País',
        title_font=dict(family="Roboto", size=20, color="#1A73E8"),
        font=dict(family="Roboto", color="#333333"),
        width=800,
        height=400,
        margin=dict(l=150, r=50, t=50, b=50),
        plot_bgcolor='white',
        paper_bgcolor='white',
        yaxis=dict(autorange="reversed")
    )

    explanation = """
    ### 🌍 Análisis por País
    - Mapa interactivo: Pasa el mouse sobre los países para ver su esperanza de vida promedio.
    - Gráfico: Top 10 países con mayor esperanza de vida.
    - Menú desplegable: Selecciona un país para ver detalles.
    """
    return explanation, map_fig, top_10_fig

# Función para actualizar información del país
def update_country_info(selected_country):
    if not selected_country or selected_country == "Selecciona un país":
        return "Selecciona un país para ver los detalles."
    country_data = country_avg[country_avg['Country'] == selected_country]
    if country_data.empty:
        return f"No se encontraron datos para {selected_country}."
    life_expectancy = country_data['Life expectancy'].iloc[0]
    return f"**País:** {selected_country}<br>**Esperanza de Vida Promedio:** {life_expectancy:.2f} años"

# 13. Creación de la Interfaz de Gradio
Creamos la interfaz con un diseño moderno y funcional.

In [13]:
# Precalcular análisis por país
country_explanation_text, country_map_fig, top_10_fig = get_country_analysis()
country_choices = ["Selecciona un país"] + country_avg['Country'].tolist()

# Crear interfaz
with gr.Blocks(css="""
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap');
    .gradio-container {
        background: linear-gradient(135deg, #F5F7FA 0%, #FFFFFF 100%);
        font-family: 'Roboto', sans-serif;
        color: #333333;
        display: flex;
        flex-direction: row;
        min-height: 100vh;
        margin: 0;
    }
    .gr-tabs {
        width: 180px;
        background-color: #1A73E8;
        padding: 10px 0;
        box-shadow: 2px 0 10px rgba(0, 0, 0, 0.1);
        position: fixed;
        height: 100%;
        overflow-y: auto;
        z-index: 1000;
    }
    .gr-tab-item {
        background-color: transparent !important;
        color: #FFFFFF !important;
        font-family: 'Roboto', sans-serif !important;
        font-weight: 500 !important;
        font-size: 14px !important;
        padding: 10px 15px !important;
        border: none !important;
        text-align: left !important;
        border-left: 3px solid transparent !important;
        display: block !important;
        width: 100% !important;
    }
    .gr-tab-item:hover {
        background-color: #1557B0 !important;
        border-left: 3px solid #FFFFFF !important;
    }
    .gr-tab-item-selected {
        background-color: #1557B0 !important;
        color: #FFFFFF !important;
        border-left: 3px solid #FFFFFF !important;
    }
    .gr-tab-content {
        margin-left: 180px;
        padding: 20px;
        flex-grow: 1;
        background-color: #FFFFFF;
        min-height: 100vh;
    }
    h1, h2, h3 {
        font-family: 'Roboto', sans-serif;
        color: #1A73E8;
        text-align: center;
        margin-bottom: 15px;
    }
    .tab {
        background-color: #FFFFFF;
        border-radius: 10px;
        padding: 20px;
        box-shadow: 0 3px 15px rgba(0, 0, 0, 0.05);
        margin: 10px 0;
        border: 1px solid #E0E0E0;
    }
    button {
        background-color: #1A73E8 !important;
        color: #FFFFFF !important;
        font-family: 'Roboto', sans-serif !important;
        font-weight: 500 !important;
        border: none !important;
        border-radius: 6px !important;
        padding: 10px 20px !important;
    }
    button:hover {
        background-color: #1557B0 !important;
    }
    .gr-input, .gr-textbox, .gr-dropdown {
        background-color: #FFFFFF !important;
        color: #333333 !important;
        border: 1px solid #E0E0E0 !important;
        border-radius: 6px !important;
        padding: 8px !important;
        font-family: 'Roboto', sans-serif !important;
    }
    .gr-input:focus, .gr-textbox:focus, .gr-dropdown:focus {
        border-color: #1A73E8 !important;
        box-shadow: 0 0 4px rgba(26, 115, 232, 0.2) !important;
    }
    .gr-plot {
        background-color: #FFFFFF !important;
        border-radius: 10px !important;
        padding: 10px !important;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05) !important;
    }
    .map-container {
        display: flex;
        justify-content: space-between;
        align-items: flex-start;
        width: 100%;
        gap: 15px;
    }
    .info-box {
        width: 30%;
        padding: 10px;
        background-color: #F5F5F5;
        border: 1px solid #E0E0E0;
        border-radius: 6px;
        font-family: 'Roboto', sans-serif;
        color: #333333;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05);
    }
    .gr-markdown {
        font-family: 'Roboto', sans-serif !important;
        color: #555555 !important;
    }
""") as demo:
    gr.Markdown("# 📊 Análisis de la Esperanza de Vida (OMS)")
    gr.Markdown("Explora datos globales de esperanza de vida con una interfaz moderna e interactiva, impulsada por IA.")

    with gr.Tabs():
        with gr.TabItem("📊 Estadísticas"):
            gr.Markdown("### Información General de los Datos")
            stats_button = gr.Button("Explorar Estadísticas")
            stats_output = gr.HTML(label="Estadísticas Descriptivas")
            stats_explanation = gr.Markdown()
            hist_plot = gr.Plot(label="Histograma de la Esperanza de Vida")
            hist_explanation = gr.Markdown()
            box_plot = gr.Plot(label="Boxplot de Variables Clave")
            boxplot_explanation_output = gr.Markdown()
            stats_button.click(
                get_dataset_stats,
                outputs=[stats_output, stats_explanation, hist_plot, hist_explanation, box_plot, boxplot_explanation_output]
            )

        with gr.TabItem("🤖 Modelo"):
            gr.Markdown("### Sobre el Modelo de Regresión Lineal")
            gr.Markdown("Este modelo predice la esperanza de vida basado en variables clave como mortalidad, escolarización, PIB, y más.")
            coef_button = gr.Button("Explorar Detalles")
            coef_output = gr.HTML(label="Coeficientes del Modelo")
            coef_explanation = gr.Markdown()
            coef_plot = gr.Plot(label="Importancia de las Variables")
            metrics_output = gr.Textbox(label="Métricas del Modelo")
            metrics_explanation = gr.Markdown()
            scatter_plot = gr.Plot(label="Predicciones vs Valores Reales")
            scatter_explanation = gr.Markdown()
            coef_button.click(
                get_model_details,
                outputs=[coef_output, coef_explanation, coef_plot, metrics_output, metrics_explanation, scatter_plot, scatter_explanation]
            )

        with gr.TabItem("🔮 Predicciones"):
            gr.Markdown("### Predice la Esperanza de Vida")
            gr.Markdown("Ingresa valores para las variables y descubre la esperanza de vida predicha.")
            with gr.Row():
                with gr.Column():
                    year = gr.Slider(2000, 2039, step=1, label="Año",
                                     info="El año para el que haces la predicción. Impacto: leve.")
                    adult_mortality = gr.Number(label="Mortalidad Adulta (por 1000)",
                                                info="Tasa de mortalidad de adultos. Impacto: Negativo. Rango: 1-723, promedio: 164.")
                    infant_deaths = gr.Number(label="Muertes Infantiles (por 1000)",
                                              info="Muertes de niños <1 año. Impacto: Negativo. Rango: 0-250, promedio: 41.")
                    alcohol = gr.Number(label="Consumo de Alcohol (litros per cápita)",
                                        info="Consumo promedio de alcohol. Impacto: Negativo. Rango: 0-17, promedio: 4.6.")
                    percentage_expenditure = gr.Number(label="Porcentaje de Gasto en Salud",
                                                       info="Porcentaje del PIB en salud. Impacto: Positivo. Rango: 0-10000, promedio: 738.")
                    hepatitis_b = gr.Number(label="Cobertura Hepatitis B (%)",
                                            info="Porcentaje de vacunación. Impacto: Positivo. Rango: 6-99, promedio: 83.")
                    measles = gr.Number(label="Casos de Sarampión",
                                        info="Casos por 1000 personas. Impacto: Negativo. Rango: 0-212183, promedio: 2421.")
                    bmi = gr.Number(label="Índice de Masa Corporal (BMI)",
                                    info="IMC promedio. Impacto: Depende. Rango: 1-87, promedio: 38.")
                    under_five_deaths = gr.Number(label="Muertes Menores de 5 años (por 1000)",
                                                  info="Muertes de niños <5 años. Impacto: Negativo. Rango: 0-397, promedio: 56.")
                    polio = gr.Number(label="Cobertura Polio (%)",
                                      info="Porcentaje de vacunación. Impacto: Positivo. Rango: 6-99, promedio: 82.")
                with gr.Column():
                    total_expenditure = gr.Number(label="Gasto Total en Salud (%)",
                                                 info="Porcentaje del presupuesto en salud. Impacto: Positivo. Rango: 1-17, promedio: 6.")
                    diphtheria = gr.Number(label="Cobertura Difteria (%)",
                                           info="Porcentaje de vacunación. Impacto: Positivo. Rango: 6-99, promedio: 82.")
                    hiv_aids = gr.Number(label="Prevalencia VIH/SIDA (por 1000)",
                                         info="Tasa de VIH/SIDA. Impacto: Negativo. Rango: 0.1-50, promedio: 1.7.")
                    gdp = gr.Number(label="PIB",
                                    info="Producto Interno Bruto. Impacto: Positivo. Rango: 1-119172, promedio: 14194.")
                    population = gr.Number(label="Población",
                                           info="Población total. Impacto: Leve. Rango: 34-1.3 mil millones, promedio: 1.3 millones.")
                    thinness_1_19_years = gr.Number(label="Delgadez 1-19 años (%)",
                                                    info="Porcentaje con bajo peso. Impacto: Negativo. Rango: 0.1-27, promedio: 5.9.")
                    thinness_5_9_years = gr.Number(label="Delgadez 5-9 años (%)",
                                                   info="Porcentaje con bajo peso. Impacto: Negativo. Rango: 0.1-28, promedio: 6.")
                    income_composition = gr.Number(label="Composición de Ingresos (0-1)",
                                                   info="Distribución de ingresos. Impacto: Positivo. Rango: 0.1-0.95, promedio: 0.67.")
                    schooling = gr.Number(label="Años de Escolarización",
                                          info="Promedio de escolarización. Impacto: Positivo. Rango: 2-21, promedio: 12.7.")
                    status_developed = gr.Dropdown([0, 1], label="Estado (0=Developing, 1=Developed)",
                                                   info="País desarrollado (1) o en desarrollo (0). Impacto: Positivo.")
            predict_button = gr.Button("Predecir")
            predict_output = gr.Textbox(label="Predicción")
            predict_button.click(
                predict_life_expectancy,
                inputs=[
                    year, adult_mortality, infant_deaths, alcohol, percentage_expenditure,
                    hepatitis_b, measles, bmi, under_five_deaths, polio, total_expenditure,
                    diphtheria, hiv_aids, gdp, population, thinness_1_19_years,
                    thinness_5_9_years, income_composition, schooling, status_developed
                ],
                outputs=predict_output
            )

        with gr.TabItem("❓ Preguntas"):
            gr.Markdown("### Explora con Preguntas")
            gr.Markdown("Selecciona una pregunta predefinida o escribe la tuya propia.")
            question_dropdown = gr.Dropdown(choices=predefined_questions, label="Selecciona una Pregunta")
            question_input = gr.Textbox(label="O Escribe tu Pregunta", placeholder="Ejemplo: ¿Cómo afecta la mortalidad adulta?")
            question_button = gr.Button("Enviar Pregunta")
            question_output = gr.Textbox(label="Respuesta del Sistema")
            def handle_question(dropdown, text_input):
                question = text_input if text_input else dropdown
                return query_huggingface_pipeline(question)
            question_button.click(
                handle_question,
                inputs=[question_dropdown, question_input],
                outputs=question_output
            )

        with gr.TabItem("🌍 Países"):
            gr.Markdown("### Esperanza de Vida por Región")
            country_explanation = gr.Markdown(value=country_explanation_text)
            with gr.Row():
                country_map = gr.Plot(label="Mapa Interactivo", value=country_map_fig)
                country_info = gr.HTML(label="Información del País", value="Selecciona un país para ver los detalles.")
            top_10_plot = gr.Plot(label="Top 10 Países con Mayor Esperanza de Vida", value=top_10_fig)
            country_dropdown = gr.Dropdown(choices=country_choices, label="Selecciona un País", value="Selecciona un país")
            country_dropdown.select(
                update_country_info,
                inputs=[country_dropdown],
                outputs=[country_info]
            )

# Lanzar interfaz
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5f7a7e3259dc92d16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# Precalcular análisis por país (asumiendo que estas funciones ya están definidas)
country_explanation_text, country_map_fig, top_10_fig = get_country_analysis()
country_choices = ["Selecciona un país"] + country_avg['Country'].tolist()

# Definir el tema y el CSS personalizado con texto en blanco
with gr.Blocks(css="""
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600;700&display=swap');

    /* Estilo general del contenedor */
    .gradio-container {
        background: linear-gradient(145deg, #1e1e2f 0%, #2a2a40 100%);
        font-family: 'Poppins', sans-serif;
        color: #FFFFFF !important; /* Color blanco para todo el texto dentro del contenedor */
        min-height: 100vh;
        margin: 0;
        padding: 0;
        overflow-x: hidden;
    }

    /* Encabezado principal */
    h1 {
        font-size: 2.5em;
        font-weight: 700;
        color: #00d4ff;
        text-align: center;
        margin: 30px 0;
        text-shadow: 0 2px 4px rgba(0, 212, 255, 0.3);
    }

    h2 {
        font-size: 1.8em;
        font-weight: 600;
        color: #00d4ff;
        margin-bottom: 15px;
    }

    h3 {
        font-size: 1.3em;
        font-weight: 500;
        color: #b0b0ff;
        margin-bottom: 10px;
    }

    /* Estilo de las pestañas */
    .gr-tabs {
        background: #25253a;
        border-radius: 15px;
        box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3);
        margin: 20px auto;
        width: 90%;
        max-width: 1200px;
    }

    .gr-tab-item {
        background: #2e2e45 !important;
        color: #FFFFFF !important; /* Texto blanco en las pestañas */
        font-family: 'Poppins', sans-serif !important;
        font-weight: 500 !important;
        font-size: 16px !important;
        padding: 15px 25px !important;
        border-radius: 10px !important;
        margin: 5px !important;
        transition: all 0.3s ease !important;
    }

    .gr-tab-item:hover {
        background: #00d4ff !important;
        color: #1e1e2f !important;
        transform: scale(1.05);
    }

    .gr-tab-item-selected {
        background: #00d4ff !important;
        color: #1e1e2f !important;
        box-shadow: 0 3px 10px rgba(0, 212, 255, 0.5) !important;
    }

    /* Contenido de las pestañas */
    .gr-tab-content {
        padding: 30px;
        background: #2e2e45;
        border-radius: 15px;
        box-shadow: 0 5px 20px rgba(0, 0, 0, 0.3);
        margin: 20px auto;
        width: 90%;
        max-width: 1200px;
    }

    /* Estilo de los botones */
    button {
        background: linear-gradient(90deg, #00d4ff 0%, #007bff 100%) !important;
        color: #1e1e2f !important;
        font-family: 'Poppins', sans-serif !important;
        font-weight: 600 !important;
        font-size: 16px !important;
        border: none !important;
        border-radius: 25px !important;
        padding: 12px 30px !important;
        transition: all 0.3s ease !important;
        box-shadow: 0 3px 10px rgba(0, 212, 255, 0.3) !important;
    }

    button:hover {
        background: linear-gradient(90deg, #007bff 0%, #00d4ff 100%) !important;
        transform: translateY(-3px);
        box-shadow: 0 5px 15px rgba(0, 212, 255, 0.5) !important;
    }

    /* Estilo de los inputs y dropdowns */
    .gr-input, .gr-textbox, .gr-dropdown, .gr-number, .gr-slider {
        background: #3a3a55 !important;
        color: #FFFFFF !important; /* Texto blanco en los inputs */
        border: 1px solid #4a4a70 !important;
        border-radius: 10px !important;
        padding: 12px !important;
        font-family: 'Poppins', sans-serif !important;
        font-size: 14px !important;
        transition: all 0.3s ease !important;
    }

    .gr-input:focus, .gr-textbox:focus, .gr-dropdown:focus, .gr-number:focus, .gr-slider:focus {
        border-color: #00d4ff !important;
        box-shadow: 0 0 8px rgba(0, 212, 255, 0.3) !important;
    }

    /* Estilo de las etiquetas (labels) */
    .gr-label, .gr-input-label, .gr-textbox-label, .gr-dropdown-label, .gr-number-label, .gr-slider-label {
        color: #FFFFFF !important; /* Texto blanco para las etiquetas */
        font-family: 'Poppins', sans-serif !important;
        font-weight: 500 !important;
        font-size: 14px !important;
    }

    /* Estilo de los gráficos */
    .gr-plot {
        background: #3a3a55 !important;
        border-radius: 15px !important;
        padding: 15px !important;
        box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3) !important;
        transition: transform 0.3s ease;
    }

    .gr-plot:hover {
        transform: translateY(-5px);
    }

    /* Estilo de las tarjetas */
    .card {
        background: #3a3a55;
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3);
        margin: 15px 0;
        transition: transform 0.3s ease;
    }

    .card:hover {
        transform: translateY(-5px);
    }

    /* Estilo de los contenedores de mapas */
    .map-container {
        display: flex;
        justify-content: space-between;
        align-items: flex-start;
        gap: 20px;
        margin: 20px 0;
    }

    .info-box {
        width: 35%;
        background: #454570;
        border-radius: 10px;
        padding: 15px;
        box-shadow: 0 3px 10px rgba(0, 0, 0, 0.3);
        color: #FFFFFF !important; /* Texto blanco en la caja de información */
        font-size: 14px;
    }

    /* Estilo del texto en Markdown */
    .gr-markdown {
        font-family: 'Poppins', sans-serif !important;
        color: #FFFFFF !important; /* Texto blanco para todos los Markdown */
        font-size: 14px !important;
    }

    /* Forzar que el texto dentro de Markdown sea blanco, incluso si está en listas o párrafos */
    .gr-markdown p, .gr-markdown li, .gr-markdown strong, .gr-markdown em {
        color: #FFFFFF !important;
    }

    /* Estilo del texto en HTML y Textbox */
    .gr-html, .gr-textbox {
        color: #FFFFFF !important; /* Texto blanco en HTML y Textbox */
        font-family: 'Poppins', sans-serif !important;
    }

    /* Forzar que el texto dentro de HTML sea blanco */
    .gr-html p, .gr-html li {
        color: #FFFFFF !important;
    }
""") as demo:
    # Título principal
    gr.Markdown("# 🌍 Análisis de la Esperanza de Vida (OMS)")
    # Asegurarse de que este texto sea blanco
    gr.Markdown(
        "**Explora datos globales de esperanza de vida con una interfaz moderna e interactiva, impulsada por IA.**",
        elem_classes=["gr-markdown"]
    )

    # Pestañas principales
    with gr.Tabs():
        # Pestaña de Estadísticas
        with gr.TabItem("📈 Estadísticas"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Información General de los Datos", elem_classes=["gr-markdown"])
                    stats_button = gr.Button("Explorar Estadísticas")
                    with gr.Row():
                        stats_output = gr.HTML(label="Estadísticas Descriptivas", elem_classes=["card"])
                        stats_explanation = gr.Markdown(elem_classes=["gr-markdown"])
            with gr.Row():
                hist_plot = gr.Plot(label="Histograma de la Esperanza de Vida", elem_classes=["card"])
                box_plot = gr.Plot(label="Boxplot de Variables Clave", elem_classes=["card"])
            hist_explanation = gr.Markdown(elem_classes=["gr-markdown"])
            boxplot_explanation_output = gr.Markdown(elem_classes=["gr-markdown"])
            stats_button.click(
                get_dataset_stats,
                outputs=[stats_output, stats_explanation, hist_plot, hist_explanation, box_plot, boxplot_explanation_output]
            )

        # Pestaña del Modelo
        with gr.TabItem("🤖 Modelo"):
            gr.Markdown("### Detalles del Modelo de Regresión Lineal", elem_classes=["gr-markdown"])
            gr.Markdown("Explora cómo el modelo predice la esperanza de vida basado en variables clave.", elem_classes=["gr-markdown"])
            coef_button = gr.Button("Ver Detalles del Modelo")
            with gr.Row():
                coef_output = gr.HTML(label="Coeficientes del Modelo", elem_classes=["card"])
                coef_plot = gr.Plot(label="Importancia de las Variables", elem_classes=["card"])
            coef_explanation = gr.Markdown(elem_classes=["gr-markdown"])
            with gr.Row():
                metrics_output = gr.Textbox(label="Métricas del Modelo", elem_classes=["card"])
                scatter_plot = gr.Plot(label="Predicciones vs Valores Reales", elem_classes=["card"])
            metrics_explanation = gr.Markdown(elem_classes=["gr-markdown"])
            scatter_explanation = gr.Markdown(elem_classes=["gr-markdown"])
            coef_button.click(
                get_model_details,
                outputs=[coef_output, coef_explanation, coef_plot, metrics_output, metrics_explanation, scatter_plot, scatter_explanation]
            )

        # Pestaña de Predicciones
        with gr.TabItem("🔮 Predicciones"):
            gr.Markdown("### Predice la Esperanza de Vida", elem_classes=["gr-markdown"])
            gr.Markdown("Ingresa los valores y obtén una predicción personalizada.", elem_classes=["gr-markdown"])
            with gr.Row():
                with gr.Column():
                    year = gr.Slider(2000, 2039, step=1, label="Año", info="El año para el que haces la predicción.")
                    adult_mortality = gr.Number(label="Mortalidad Adulta (por 1000)", info="Tasa de mortalidad de adultos.")
                    infant_deaths = gr.Number(label="Muertes Infantiles (por 1000)", info="Muertes de niños <1 año.")
                    alcohol = gr.Number(label="Consumo de Alcohol (litros per cápita)", info="Consumo promedio de alcohol.")
                    percentage_expenditure = gr.Number(label="Porcentaje de Gasto en Salud", info="Porcentaje del PIB en salud.")
                    hepatitis_b = gr.Number(label="Cobertura Hepatitis B (%)", info="Porcentaje de vacunación.")
                    measles = gr.Number(label="Casos de Sarampión", info="Casos por 1000 personas.")
                    bmi = gr.Number(label="Índice de Masa Corporal (BMI)", info="IMC promedio.")
                    under_five_deaths = gr.Number(label="Muertes Menores de 5 años (por 1000)", info="Muertes de niños <5 años.")
                    polio = gr.Number(label="Cobertura Polio (%)", info="Porcentaje de vacunación.")
                with gr.Column():
                    total_expenditure = gr.Number(label="Gasto Total en Salud (%)", info="Porcentaje del presupuesto en salud.")
                    diphtheria = gr.Number(label="Cobertura Difteria (%)", info="Porcentaje de vacunación.")
                    hiv_aids = gr.Number(label="Prevalencia VIH/SIDA (por 1000)", info="Tasa de VIH/SIDA.")
                    gdp = gr.Number(label="PIB", info="Producto Interno Bruto.")
                    population = gr.Number(label="Población", info="Población total.")
                    thinness_1_19_years = gr.Number(label="Delgadez 1-19 años (%)", info="Porcentaje con bajo peso.")
                    thinness_5_9_years = gr.Number(label="Delgadez 5-9 años (%)", info="Porcentaje con bajo peso.")
                    income_composition = gr.Number(label="Composición de Ingresos (0-1)", info="Distribución de ingresos.")
                    schooling = gr.Number(label="Años de Escolarización", info="Promedio de escolarización.")
                    status_developed = gr.Dropdown([0, 1], label="Estado (0=Developing, 1=Developed)", info="País desarrollado o en desarrollo.")
            predict_button = gr.Button("Predecir Esperanza de Vida")
            predict_output = gr.Textbox(label="Esperanza de Vida Predicha", elem_classes=["card"])
            predict_button.click(
                predict_life_expectancy,
                inputs=[
                    year, adult_mortality, infant_deaths, alcohol, percentage_expenditure,
                    hepatitis_b, measles, bmi, under_five_deaths, polio, total_expenditure,
                    diphtheria, hiv_aids, gdp, population, thinness_1_19_years,
                    thinness_5_9_years, income_composition, schooling, status_developed
                ],
                outputs=predict_output
            )

        # Pestaña de Preguntas
        with gr.TabItem("❓ Preguntas"):
            gr.Markdown("### Explora con Preguntas", elem_classes=["gr-markdown"])
            gr.Markdown("Haz preguntas predefinidas o escribe las tuyas propias.", elem_classes=["gr-markdown"])
            with gr.Row():
                question_dropdown = gr.Dropdown(choices=predefined_questions, label="Selecciona una Pregunta")
                question_input = gr.Textbox(label="O Escribe tu Pregunta", placeholder="Ejemplo: ¿Cómo afecta la mortalidad adulta?")
            question_button = gr.Button("Enviar Pregunta")
            question_output = gr.Textbox(label="Respuesta del Sistema", elem_classes=["card"])
            def handle_question(dropdown, text_input):
                question = text_input if text_input else dropdown
                return query_huggingface_pipeline(question)
            question_button.click(
                handle_question,
                inputs=[question_dropdown, question_input],
                outputs=question_output
            )

        # Pestaña de Países
        with gr.TabItem("🌍 Países"):
            gr.Markdown("### Análisis por Región", elem_classes=["gr-markdown"])
            country_explanation = gr.Markdown(value=country_explanation_text, elem_classes=["gr-markdown"])
            with gr.Row(elem_classes=["map-container"]):
                country_map = gr.Plot(label="Mapa Interactivo", value=country_map_fig, elem_classes=["card"])
                country_info = gr.HTML(label="Información del País", value="Selecciona un país para ver los detalles.", elem_classes=["info-box"])
            top_10_plot = gr.Plot(label="Top 10 Países con Mayor Esperanza de Vida", value=top_10_fig, elem_classes=["card"])
            country_dropdown = gr.Dropdown(choices=country_choices, label="Selecciona un País", value="Selecciona un país")
            country_dropdown.select(
                update_country_info,
                inputs=[country_dropdown],
                outputs=[country_info]
            )

# Lanzar la interfaz
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb37f6eca54e7a91b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 14. Limpiar el Notebook para Evitar el Error de GitHub

In [ ]:
import nbformat

# Leer el notebook actual
with open("/content/Esperanza_de_vida2000_2015.ipynb", "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Eliminar metadata.widgets de las celdas y del notebook
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]
for cell in nb.cells:
    if "metadata" in cell and "widgets" in cell.metadata:
        del cell.metadata["widgets"]

# Guardar el notebook limpio
with open("/content/Esperanza_de_vida2000-2015_clean.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Notebook limpio guardado como: /content/Esperanza_de_vida2000-2015_clean.ipynb")

# 15. (Opcional) Limpiar las Salidas para Reducir el Tamaño

In [ ]:
import nbformat

with open("/content/Esperanza_de_vida2000-2015_clean.ipynb", "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
    if "outputs" in cell:
        cell["outputs"] = []

with open("/content/Esperanza_de_vida2000-2015_clean_no_outputs.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Notebook limpio sin salidas guardado como: /content/Esperanza_de_vida2000-2015_clean_no_outputs.ipynb")

Notebook limpio sin salidas guardado como: /content/Esperanza_de_vida2000-2015_clean_no_outputs.ipynb


# 16. Crear una Celda para Subir a GitHub

In [ ]:
from google.colab import userdata
import requests
import base64

# Configuración
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')  # Obtener el token de los secretos
GITHUB_USERNAME = "Arcane-jinx"  # Tu nombre de usuario de GitHub
REPO_NAME = "Dataset"  # Nombre del repositorio
FILE_PATH = "Esperanza_de_vida2000-2015_clean.ipynb"  # Ruta del archivo en el repositorio
COMMIT_MESSAGE = "Subir notebook limpio desde Google Colab"

# Definir headers
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# Verificar el token
user_url = "https://api.github.com/user"
response = requests.get(user_url, headers=headers)
if response.status_code != 200:
    print(f"Error con el token: {response.text}")
    raise Exception("Token inválido o sin permisos")
else:
    print("Token válido. Usuario:", response.json()['login'])

# Leer el archivo limpio
try:
    with open("/content/Esperanza_de_vida2000-2015_clean.ipynb", "rb") as f:
        content = f.read()
except FileNotFoundError:
    raise Exception("El archivo /content/Esperanza_de_vida2000-2015_clean.ipynb no existe. Asegúrate de haberlo creado.")

# Codificar el contenido en base64
encoded_content = base64.b64encode(content).decode('utf-8')

# Verificar si el repositorio existe
repo_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}"
response = requests.get(repo_url, headers=headers)
if response.status_code == 200:
    print(f"Repositorio {REPO_NAME} ya existe.")
elif response.status_code == 404:
    # Crear el repositorio si no existe
    create_repo_url = "https://api.github.com/user/repos"  # URL correcta para crear repositorios
    repo_data = {
        "name": REPO_NAME,
        "description": "Análisis de esperanza de vida con dataset de la OMS",
        "auto_init": True
    }
    response = requests.post(create_repo_url, headers=headers, json=repo_data)
    if response.status_code == 201:
        print(f"Repositorio {REPO_NAME} creado exitosamente.")
    else:
        print(f"Error al crear el repositorio: {response.text}")
        raise Exception("No se pudo crear el repositorio")
else:
    print(f"Error al verificar el repositorio: {response.text}")
    raise Exception("No se pudo verificar el repositorio")

# Subir el archivo al repositorio
file_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/contents/{FILE_PATH}"
file_data = {
    "message": COMMIT_MESSAGE,
    "content": encoded_content
}
response = requests.put(file_url, headers=headers, json=file_data)
if response.status_code in [201, 200]:
    print(f"Archivo subido exitosamente: {response.json()['content']['html_url']}")
else:
    print(f"Error al subir el archivo: {response.text}")
    raise Exception("No se pudo subir el archivo")

# 17. (Opcional) Subir el Dataset y Crear un README

In [ ]:
from google.colab import userdata
import requests
import base64
import json

# Configuración
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')  # Obtener el token de los secretos
GITHUB_USERNAME = "Arcane-jinx"
REPO_NAME = "Dataset"
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# Verificar si el dataset ya existe
dataset_path = "Esperanza_de_vida2000-2015.csv"
dataset_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/contents/{dataset_path}"
response = requests.get(dataset_url, headers=headers)
if response.status_code == 200:
    print("El dataset ya existe en el repositorio.")
else:
    # Si el dataset no existe, subirlo
    try:
        with open("/content/Esperanza_de_vida2000-2015.csv", "rb") as f:
            content = f.read()
        encoded_content = base64.b64encode(content).decode('utf-8')
        file_data = {
            "message": "Subir dataset",
            "content": encoded_content
        }
        response = requests.put(dataset_url, headers=headers, json=file_data)
        if response.status_code in [201, 200]:
            print(f"Dataset subido exitosamente: {response.json()['content']['html_url']}")
        else:
            print(f"Error al subir el dataset: {response.text}")
    except FileNotFoundError:
        print("El archivo /content/Esperanza_de_vida2000-2015.csv no existe en Colab. Asegúrate de subirlo.")

# Obtener el contenido actual del README.md (si existe)
readme_path = "README.md"
readme_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/contents/{readme_path}"
response = requests.get(readme_url, headers=headers)
sha = None
if response.status_code == 200:
    # El README ya existe, obtener su SHA para actualizarlo
    readme_data = response.json()
    sha = readme_data['sha']
    print("README.md encontrado. Se actualizará el contenido.")
else:
    print("README.md no encontrado. Se creará uno nuevo.")

# Nuevo contenido del README.md (basado en lo que mostraste en la captura)
readme_content = """# Dataset

## (OMS) Esperanza de vida

Aquí te detallo algunas ideas de predicciones que podrías realizar:

1. **Predicción de la Esperanza de Vida** (como en el código actual) Qué predecir: La esperanza de vida (Life expectancy) en función de las otras variables. Variables predictoras: Mortalidad adulta (Adult Mortality), mortalidad infantil (infant deaths), consumo de alcohol (Alcohol), gasto en salud (percentage expenditure, Total expenditure)

## Archivos
- `Esperanza_de_vida2000-2015_clean.ipynb`: Notebook con el análisis completo. Incluye gráficos estáticos y una interfaz interactiva con Gradio (ejecutar en Colab para interactividad).
- `Esperanza_de_vida2000-2015.csv`: Dataset utilizado.

## Instrucciones
1. Descarga el notebook y el dataset.
2. Abre el notebook en Google Colab.
3. Sigue las instrucciones en el notebook para instalar dependencias y ejecutar el código.
4. Para usar la interfaz interactiva, asegúrate de ejecutar la última celda (interfaz de Gradio).

## Notas
- El mapa interactivo y las visualizaciones estáticas se renderizan correctamente en GitHub.
- La interfaz de Gradio requiere ejecución en Colab o localmente.
"""

# Codificar el contenido del README
encoded_readme = base64.b64encode(readme_content.encode('utf-8')).decode('utf-8')

# Actualizar o crear el README.md
readme_data = {
    "message": "Actualizar README.md",
    "content": encoded_readme
}
if sha:
    readme_data["sha"] = sha  # Incluir el SHA para actualizar el archivo existente

response = requests.put(readme_url, headers=headers, json=readme_data)
if response.status_code in [201, 200]:
    print("README.md actualizado exitosamente.")
else:
    print(f"Error al actualizar el README: {response.text}")

El dataset ya existe en el repositorio.
README.md encontrado. Se actualizará el contenido.
README.md actualizado exitosamente.


# 18. Actualizacion del GitHub

In [ ]:
from google.colab import userdata
import requests
import base64

# Configuración
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')  # Obtener el token de los secretos
GITHUB_USERNAME = "Arcane-jinx"
REPO_NAME = "Dataset"
FILE_PATH = "Esperanza_de_vida2000-2015_clean.ipynb"  # Nombre del archivo en el repositorio
COMMIT_MESSAGE = "Actualizar notebook desde Google Colab"

# Definir headers
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# Verificar si el archivo ya existe en el repositorio y obtener su SHA
file_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/contents/{FILE_PATH}"
response = requests.get(file_url, headers=headers)
sha = None
if response.status_code == 200:
    # El archivo ya existe, obtener su SHA para actualizarlo
    file_data = response.json()
    sha = file_data['sha']
    print(f"Archivo {FILE_PATH} encontrado en el repositorio. Se actualizará.")
else:
    print(f"Archivo {FILE_PATH} no encontrado en el repositorio. Se creará uno nuevo.")

# Leer el archivo local desde Colab
try:
    with open("/content/Esperanza_de_vida2000-2015_clean.ipynb", "rb") as f:
        content = f.read()
except FileNotFoundError:
    raise Exception("El archivo /content/Esperanza_de_vida2000-2015_clean.ipynb no existe en Colab. Asegúrate de que esté disponible.")

# Codificar el contenido en base64
encoded_content = base64.b64encode(content).decode('utf-8')

# Preparar los datos para la solicitud
update_data = {
    "message": COMMIT_MESSAGE,
    "content": encoded_content
}
if sha:
    update_data["sha"] = sha  # Incluir el SHA para actualizar el archivo existente

# Actualizar o crear el archivo en GitHub
response = requests.put(file_url, headers=headers, json=update_data)
if response.status_code in [201, 200]:
    print(f"Archivo {FILE_PATH} actualizado exitosamente: {response.json()['content']['html_url']}")
else:
    print(f"Error al actualizar el archivo: {response.text}")
    raise Exception("No se pudo actualizar el archivo")

Archivo Esperanza_de_vida2000-2015_clean.ipynb encontrado en el repositorio. Se actualizará.
Archivo Esperanza_de_vida2000-2015_clean.ipynb actualizado exitosamente: https://github.com/Arcane-jinx/Dataset/blob/main/Esperanza_de_vida2000-2015_clean.ipynb
